# ATTENTION !!!!
### Nothing has been tested yet, so it could (but I don't think) have some bug. Carefull when you implement it.

# The load pairs function
I just change the interpolation and suppr the .astype() useless because imresize change it (and my script deal with this problem alone)

In [ ]:
def load_pairs_from_preprocessed(match_path, mismatch_path):
    images_match = dict()

    for ID in range(94):
        for pic in range(2):
            image_temp = np.zeros((image_size,image_size,num_layer))
            for layer in range(num_layer):
                fname = str(ID+1)+'_'+str(pic)+'_'+str(layer)+'.png'
                image_temp[:,:,layer] = imresize(imread(match_path+fname),[image_size,image_size],interp='bicubic')
            images_match[str(ID+1)+'_'+str(pic)] = image_temp
    
    images_mismatch = dict()

    for ID in range(94):
        for pic in range(2):
            image_temp = np.zeros((image_size,image_size,num_layer))
            for layer in range(num_layer):
                fname = str(ID+1)+'_'+str(pic)+'_'+str(layer)+'.png'
                image_temp[:,:,layer] = imresize(imread(mismatch_path+fname),[image_size,image_size],interp='bicubic')
            images_mismatch[str(ID+1)+'_'+str(pic)] = image_temp
            
    return images_match, images_mismatch

# Batch without duplicate
I just add the 2 lines after "# To use only one each pairs" and a security to don't create batch bigger than we can

In [ ]:
def generate_batch_pairs_from_preprocessed(images_match, images_mismatch, num, image_size):
        
    x = np.zeros((num, image_size*image_size*num_layer))
    x_p = np.zeros((num, image_size*image_size*num_layer))
    y = np.ones((num, 1))*-1
    match = int(np.round(num*0.5))
    mis_match = num - match

    keys_mismatch = images_mismatch.keys()

    if mis_match > len(keys_mismatch):
        mis_match = len(keys_mismatch)
        print 'Number of Mismatch wanted to big !!!!'

    for i in range(mis_match):
        ID = random.choice(keys_mismatch)[:-1]
        img = images_mismatch[ID + '0']
        img_p = images_mismatch[ID + '1']
        # To use only one each pairs
        keys_match.remove(img)
        keys_match.remove(img_p)
        x[i,:] = np.reshape(img, (1,image_size*image_size*num_layer))
        x_p[i,:] = np.reshape(img_p, (1,image_size*image_size*num_layer))
        y[i] = 0

    keys_match = images_match.keys()

    if match > len(keys_match):
        match = len(keys_match)
        print 'Number of Match wanted to big !!!!'

    for i in range(match):
        ID = random.choice(keys_match)[:-1]
        img = images_match[ID + '0']
        img_p = images_match[ID + '1']
        # To use only each pairs once
        keys_match.remove(img)
        keys_match.remove(img_p)
        x[mis_match+i,:] = np.reshape(img, (1,image_size*image_size*num_layer))
        x_p[mis_match+i,:] = np.reshape(img_p, (1,image_size*image_size*num_layer))
        y[mis_match+i] = 1
    
    return [x, x_p, y]

# Suffle a batch
For suffle a batch ^^

In [ ]:
def suffle_all(x, x_p, y):
    x_shuffle = []
    x_p_shuffle = []
    y_shuffle = []
    
    shuffle = random.shuffle(range(len(x)))
    
    for i in shuffle:
        x_shuffle.append(x[i])
        x_p_shuffle.append(x_p[i])
        y_shuffle.append(y[i])

    return [x_shuffle, x_p_shuffle, y_shuffle]

# Create a eval batch different from the train set
Same algo as before but using the pop function

In [ ]:
def eval_and_train(images_match, images_mismatch, num, image_size):

    x = np.zeros((num, image_size*image_size*num_layer))
    x_p = np.zeros((num, image_size*image_size*num_layer))
    y = np.ones((num, 1))*-1
    match = int(np.round(num*0.5))
    mis_match = num - match

    keys_mismatch = images_mismatch.keys()
    for i in range(mis_match):
        ID = random.choice(keys_mismatch)[:-1]
        img = images_mismatch.pop(ID + '0')
        img_p = images_mismatch.pop(ID + '1')
        # To use only one each pairs
        keys_match.remove(img)
        keys_match.remove(img_p)
        x[i,:] = np.reshape(img, (1,image_size*image_size*num_layer))
        x_p[i,:] = np.reshape(img_p, (1,image_size*image_size*num_layer))
        y[i] = 0

    keys_match = images_match.keys()
    for i in range(match):
        ID = random.choice(keys_match)[:-1]
        img = images_match.pop(ID + '0')
        img_p = images_match.pop(ID + '1')
        # To use only one each pairs
        keys_match.remove(img)
        keys_match.remove(img_p)
        x[mis_match+i,:] = np.reshape(img, (1,image_size*image_size*num_layer))
        x_p[mis_match+i,:] = np.reshape(img_p, (1,image_size*image_size*num_layer))
        y[mis_match+i] = 1
    
    return ((x, x_p, y), (images_match, images_mismatch))